In [3]:
# pip install pytorch-tabnet

In [4]:
# pip install torch --upgrade

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from pytorch_tabnet.multitask import TabNetMultiTaskClassifier
from pytorch_tabnet.metrics import Metric

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

import datetime
from datetime import datetime as dt

import requests
import json
import csv

import warnings
warnings.filterwarnings('ignore')

In [2]:
df1 = pd.read_csv('{파일경로}/lifelog_data_2018.csv')
df2 = pd.read_csv('{파일경로}/lifelog_data_2019.csv')
df3 = pd.read_csv('{파일경로}/lifelog_data_2020.csv')

In [5]:
def engineering(n, d):
    # 성별 인코딩
    d['gender'] = d['gender'].apply(lambda x:0 if x == 'M' else 1)
    # 날짜 변환
    d['date'] = pd.to_datetime(d['date'])

    # 스트레스(target)
    d['pmStress'] = d['pmStress'] - 1
    # XGBoost의 경우는 0부터 시작 > 나중에 좀 더 찾아보기
    # Stacking을 위해 모든 모델 pmStress 범위 변경
    
    # 감정변화비율 = 오후감정/오전감정
    d['emotionChangeRate'] = d['pmEmotion'] / d['amEmotion']
    # 긍정변화평균 = emotion1~7의 평균
    d['positiveMean'] = d.filter(regex='Positive').mean(axis=1)
    # 긴장변화평균 = emotion1~7의 평균
    d['tensionnMean'] = d.filter(regex='Tension').mean(axis=1)
    
    # 긍정비율 = Positive5~7 / 1~3 > 4는 중앙값으로 진행하지 않음
    d['positiveRate'] = d[['emotionPositive5', 'emotionPositive6', 'emotionPositive7']].sum(axis=1) / d[['emotionPositive1', 'emotionPositive2', 'emotionPositive3']].sum(axis=1)
    # 긴장비율 = Tension5~7 / 1~3 > 4는 중앙값으로 진행하지 않음
    d['tensionRate'] = d[['emotionTension5', 'emotionTension6', 'emotionTension7']].sum(axis=1) / d[['emotionTension1', 'emotionTension2', 'emotionTension3']].sum(axis=1)
    
    # 긍정감정의 임시 테이블
    pos_temp = d.filter(regex='Positive')
    # 가장 높은 긍정감정의 숫자(1~7)
    d['topPositive'] = pos_temp.idxmax(axis=1).apply(lambda x:int(x[-1]))
    # 가장 낮은 긍정감정의 숫자(1~7)
    d['botPositive'] = pos_temp.idxmin(axis=1).apply(lambda x:int(x[-1]))
    
    # 긍정 수치 1~7 * count 수
    for p, c in enumerate(pos_temp.columns):
        d.loc[:, [c]] = d.loc[:, [c]] * (p+1)
        
    del pos_temp
    
    # 긴장감정의 임시 테이블
    ten_temp = d.filter(regex='Tension')
    # 가장 높은 긴장감정의 숫자(1~7)
    d['topTension'] = ten_temp.idxmax(axis=1).apply(lambda x:int(x[-1]))
    # 가장 낮은 긴장감정의 숫자(1~7)
    d['botTension'] = ten_temp.idxmin(axis=1).apply(lambda x:int(x[-1]))
    
    # 긴장 수치 1~7 * count 수
    for t, c in enumerate(ten_temp.columns):
        d.loc[:, [c]] = d.loc[:, [c]] * (t+1)
    
    del ten_temp
    
    # 수치 반영 긍정 평균
    d['positiveWMean'] = d.filter(regex='Positive').mean(axis=1)
    # 수치 반영 긴장 평균
    d['tensionWMean'] = d.filter(regex='Tension').mean(axis=1)
    
    # 수치 반영 긍정 비율
    d['positiveWRate'] = d[['emotionPositive5', 'emotionPositive6', 'emotionPositive7']].sum(axis=1) / d[['emotionPositive1', 'emotionPositive2', 'emotionPositive3']].sum(axis=1)
    # 수치 반영 긴장 비율
    d['tensionWRate'] = d[['emotionTension5', 'emotionTension6', 'emotionTension7']].sum(axis=1) / d[['emotionTension1', 'emotionTension2', 'emotionTension3']].sum(axis=1)
    
    # 최고 수치를 반영한 오전 감정 = (오전 감정 * (최고 감정 + 최고 긴장)) / 14
    d['aCtPT'] = round(d['amEmotion'] * (d['topPositive'] + d['topTension']) / 14)
    # 최고 수치를 반영한 오전 컨디션 = (오전 감정 * (최고 감정 + 최고 긴장)) / 14
    d['aEtPT'] = round(d['amCondition'] * (d['topPositive'] + d['topTension']) / 14)
    # 최고 수치를 반영한 오후 감정 = (오전 감정 * (최고 감정 + 최고 긴장)) / 14
    d['pEtPT'] = round(d['pmEmotion'] * (d['topPositive'] + d['topTension']) / 14)
    
    d['aCEpEtPTm'] = (d['aCtPT'] + d['aEtPT'] + d['pEtPT'])
    
    # 긍정적인지 = 수치 반영 긍정 평균이 중앙값보다 크면 1 아니면 0
    d['positive'] = d['positiveWRate'].apply(lambda x:1 if x > d['positiveWRate'].median() else 0)
    # 부정적인지 = 수치 반영 긍정 평균이 중앙값보다 작으면 1 아니면 0
    d['negative'] = d['positiveWRate'].apply(lambda x:1 if x < d['positiveWRate'].median() else 0)
    
    # 긴장 상태인지 = 수치 반영 긴장 평균이 중앙값보다 크면 1 아니면 0
    d['aroused'] = d['tensionWRate'].apply(lambda x:1 if x > d['tensionWRate'].median() else 0)
    # 편안한 상태인지 = 수치 반영 긴장 평균이 중앙값보다 작으면 1 아니면 0
    d['relaxed'] = d['tensionWRate'].apply(lambda x:1 if x < d['tensionWRate'].median() else 0)
    
    # 활동 비율 = 자전거, 도보의 합 / 운송수단, 가만히 있기의 합
    d['activityRate'] = d[['on_bicycle', 'on_foot']].sum(axis=1) / d[['in_vehicle', 'still']].sum(axis=1)
    # 0으로 나누기된 것을 0으로 치환 > inf 값을 가짐
    d.replace([np.inf, -np.inf], 0, inplace=True)
    d.dropna(inplace=True)
    d.sample(n=d.shape[0], random_state=47)
    
    return d

In [6]:
for n, d in enumerate([df1, df2, df3]):
    globals() [f'df{n+1}'] = engineering(n, d.copy())
df = pd.concat([df1, df2, df3], axis=0)
df = df[[c for c in df.columns if c not in ['pmStress']] + ['pmStress']]
df.drop(['date', 'userId'], axis = 1, inplace = True)

In [10]:
test_accuracy_list = []

for i in range(20):
    train = pd.DataFrame()
    valid = pd.DataFrame()
    test = pd.DataFrame()

    for i in range(5):
        train = pd.concat([train, df.loc[df['pmStress'] == np.unique(df['pmStress'])[i]][:round(len(df) / 5 * 0.7)]], axis=0)
        valid = pd.concat([valid, df.loc[df['pmStress'] == np.unique(df['pmStress'])[i]][round(len(df) / 5 * 0.7):-round(len(df) / 5 * 0.1)]], axis=0)
        test = pd.concat([test, df.loc[df['pmStress'] == np.unique(df['pmStress'])[i]][-round(len(df) / 5 * 0.1):]], axis=0)

        # 동 수로 뽑을 때
    #     temp = df.loc[df['pmStress'] == np.unique(df['pmStress'])[i]][:112]
    #     train = pd.concat([train, temp[:round(112 * 0.7)]], axis=0)
    #     valid = pd.concat([valid, temp[round(112 * 0.7):-round(112 * 0.1)]], axis=0)
    #     test = pd.concat([test, temp[-round(112 * 0.1):]], axis=0)

    train = train.sample(n=train.shape[0]) # , random_state=1234)
    valid = valid.sample(n=valid.shape[0]) # , random_state=1234)
    test = test.sample(n=test.shape[0]) # , random_state=1234)
    
    df = pd.concat([train, test, valid], ignore_index=True, axis=0)
    df = df[[c for c in df.columns if c not in ['userId', 'date']]]

    # train_indices = range(390) # train.index
    # valid_indices = range(390, 505) # valid.index
    # test_indices = range(505, 560) # test.index
    train_indices = train.index
    valid_indices = valid.index
    test_indices = test.index

    nunique = df.nunique()
    types = df.dtypes

    categorical_columns = []
    categorical_dims =  {}
    for col in df.columns:
        if types[col] == 'object' or nunique[col] <= 7:
            categorical_columns.append(col)
            categorical_dims[col] = len(np.unique(df[col]))

    features = [col for col in df.columns if col not in ['pmStress']] 
    cat_idxs = [i for i, f in enumerate(features) if f in categorical_columns]
    cat_dims = [categorical_dims[f] for f in features if f in categorical_columns]

    X_train = df[features].values[train_indices]
    y_train = df['pmStress'].values[train_indices].reshape(-1, 1)

    X_valid = df[features].values[valid_indices]
    y_valid = df['pmStress'].values[valid_indices].reshape(-1, 1)

    X_test = df[features].values[test_indices]
    y_test = df['pmStress'].values[test_indices].reshape(-1, 1)

    of = optim.Adam
    op = dict(lr=0.01)
    ns = 4
    nd = 24
    g = 1.3
    ss = 10
    gm = 0.95
    mt = 'entmax'
    bs = 16
    em = 'logloss'
    ws = True
    clf = TabNetMultiTaskClassifier(optimizer_fn=of, optimizer_params=op, scheduler_fn=optim.lr_scheduler.StepLR,
    n_steps=ns, n_d=nd, n_a=nd, gamma=g,
    scheduler_params={"step_size":ss, "gamma":gm}, 
    mask_type=mt, verbose=0)

    clf.fit(X_train.copy(), y_train.copy(),
            eval_set=[(X_valid.copy(), y_valid.copy())],
            loss_fn=[torch.nn.functional.cross_entropy]*5,
            max_epochs=30, 
            patience=0,
            batch_size=bs,
            eval_metric=[em],
            warm_start=ws)
    pred_TabNet = clf.predict(X_test.copy())
    accuracy = accuracy_score(y_test.copy(), np.array(pred_TabNet.copy()[0]).astype('float').reshape(-1, 1))
    print("="*30, '\n', of, op, ns, nd, g, ss, gm, mt, bs, em, ws, '\n', 'accuracy :', accuracy)
    test_accuracy_list.append(accuracy)

# result_TabNet = pd.DataFrame({'Test Accuracy': test_accuracy_list})

 <class 'torch.optim.adam.Adam'> {'lr': 0.01} 4 24 1.3 10 0.95 entmax 16 logloss True 
 accuracy : 0.3652173913043478
 <class 'torch.optim.adam.Adam'> {'lr': 0.01} 4 24 1.3 10 0.95 entmax 16 logloss True 
 accuracy : 0.3826086956521739
 <class 'torch.optim.adam.Adam'> {'lr': 0.01} 4 24 1.3 10 0.95 entmax 16 logloss True 
 accuracy : 0.33043478260869563
 <class 'torch.optim.adam.Adam'> {'lr': 0.01} 4 24 1.3 10 0.95 entmax 16 logloss True 
 accuracy : 0.4434782608695652
 <class 'torch.optim.adam.Adam'> {'lr': 0.01} 4 24 1.3 10 0.95 entmax 16 logloss True 
 accuracy : 0.2956521739130435
 <class 'torch.optim.adam.Adam'> {'lr': 0.01} 4 24 1.3 10 0.95 entmax 16 logloss True 
 accuracy : 0.34782608695652173
 <class 'torch.optim.adam.Adam'> {'lr': 0.01} 4 24 1.3 10 0.95 entmax 16 logloss True 
 accuracy : 0.3391304347826087
 <class 'torch.optim.adam.Adam'> {'lr': 0.01} 4 24 1.3 10 0.95 entmax 16 logloss True 
 accuracy : 0.3739130434782609
 <class 'torch.optim.adam.Adam'> {'lr': 0.01} 4 24 1.3

In [85]:
y_pred_TabNet = pd.DataFrame(pred_TabNet)
y_pred_TabNet = y_pred_TabNet.transpose()
y_pred_TabNet.columns = ['TabNet']
y_pred_TabNet.to_csv('TabNet.csv')